In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Importing the Data
df = pd.read_csv(r"D:\Projects\Uber\Data\data_cleaned.csv",index_col=0)
df.head()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Ordered,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,Multiple_deliveries,Festival,City_type,Time_taken(min),city_code
0,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,2,Snack,motorcycle,0.0,No,Urban,24,INDO
1,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,2,Snack,scooter,1.0,No,Metropolitian,33,BANG
2,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,0,Drinks,motorcycle,1.0,No,Urban,26,BANG
3,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,0,Buffet,motorcycle,1.0,No,Metropolitian,21,COIMB
4,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,1,Snack,scooter,1.0,No,Metropolitian,30,CHEN


In [4]:

def extract_date_features(data):
    data['Order_Date'] = data["Order_Date"] = pd.to_datetime(data["Order_Date"])
    data['is_weekend'] = data['Order_Date'].dt.day_of_week>4
    data['month_interval'] =data['Order_Date'].apply(lambda x: "start" if x.day <=10
                                                   else ("middle" if x.day <= 20 else "end"))
extract_date_features(df)
df.head()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Ordered,Time_Order_picked,Weather_conditions,...,Vehicle_condition,Type_of_order,Type_of_vehicle,Multiple_deliveries,Festival,City_type,Time_taken(min),city_code,is_weekend,month_interval
0,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,...,2,Snack,motorcycle,0.0,No,Urban,24,INDO,True,middle
1,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,...,2,Snack,scooter,1.0,No,Metropolitian,33,BANG,False,end
2,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,...,0,Drinks,motorcycle,1.0,No,Urban,26,BANG,True,middle
3,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,...,0,Buffet,motorcycle,1.0,No,Metropolitian,21,COIMB,False,start
4,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,...,1,Snack,scooter,1.0,No,Metropolitian,30,CHEN,True,end


In [5]:
def time_difference(df):
    df['Time_Ordered'] = pd.to_timedelta(df['Time_Ordered'])
    df['Time_Order_picked'] = pd.to_timedelta(df['Time_Order_picked'])

    df['Time_Order_picked_formatted'] = df['Order_Date'] + pd.to_timedelta(np.where(df['Time_Order_picked'] < df['Time_Ordered'], 1, 0), unit='D') + df['Time_Order_picked']
    # print(df['Time_Order_picked_formatted'])

    # Calculate formatted order time
    df['Time_Ordered_formatted'] = df['Order_Date'] + df['Time_Ordered']
    # print(df['Time_Ordered_formatted'])

    # Calculate time difference in minutes
    df['order_prepare_time'] = (df['Time_Order_picked_formatted'] - df['Time_Ordered_formatted']).dt.total_seconds() / 60
    # print(df['order_prepare_time'])

    # Handle null values by filling with the median
    df['order_prepare_time'].fillna(df['order_prepare_time'].median(), inplace=True)
    # print(df['order_prepare_time'])

    # Drop all the time & date related columns
    df.drop(['Time_Ordered', 'Time_Order_picked', 'Time_Ordered_formatted', 'Time_Order_picked_formatted', 'Order_Date'], axis=1, inplace=True)

time_difference(df)
df.head()


,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,Multiple_deliveries,Festival,City_type,Time_taken(min),city_code,is_weekend,month_interval,order_prepare_time
0,37.0,4.9,22.745049,75.892471,22.765049,75.912471,Sunny,High,2,Snack,motorcycle,0.0,No,Urban,24,INDO,True,middle,15.0
1,34.0,4.5,12.913041,77.683237,13.043041,77.813237,Stormy,Jam,2,Snack,scooter,1.0,No,Metropolitian,33,BANG,False,end,5.0
2,23.0,4.4,12.914264,77.678400,12.924264,77.688400,Sandstorms,Low,0,Drinks,motorcycle,1.0,No,Urban,26,BANG,True,middle,15.0
3,38.0,4.7,11.003669,76.976494,11.053669,77.026494,Sunny,Medium,0,Buffet,motorcycle,1.0,No,Metropolitian,21,COIMB,False,start,10.0
4,32.0,4.6,12.972793,80.249982,13.012793,80.289982,Cloudy,High,1,Snack,scooter,1.0,No,Metropolitian,30,CHEN,True,end,15.0


<img src ='https://th.bing.com/th/id/OIP.EHJgnPCKMqZA5UzyHlAFlAHaFj?rs=1&pid=ImgDetMain'>

In [6]:
R =6317
def deg_to_rad(degree):
    return degree*(np.pi/180)

def distcalculate(lat1,lon1,lat2,lon2):
    d_lat = deg_to_rad(lat2-lat1)
    d_lon = deg_to_rad(lon2-lon1)
    a1 = np.sin(d_lat/2)**2 
    a2 =np.cos(deg_to_rad(lat1))* np.cos(deg_to_rad(lat2)) * np.sin(d_lon/2)**2
    a = a1 + a2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c







df['distance'] = np.nan
for i in range(len(df)):
    df.loc[i,'distance'] = distcalculate(df.loc[i, 'Restaurant_latitude'],
                                          df.loc[i, 'Restaurant_longitude'],
                                          df.loc[i, 'Delivery_location_latitude'],
                                          df.loc[i, 'Delivery_location_longitude'])

df.distance = df.distance.astype("int64")
df.distance.head()

0     2
1    20
2     1
3     7
4     6
Name: distance, dtype: int64

In [7]:
df.to_csv("../Data/data_after_feature_engineering.csv")